In [12]:
import json
import os
from rake_nltk import Rake
import networkx
# 一个图结构的相关操作包，没用过无所谓，有兴趣可以搜索学习
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import yake
from keybert import KeyBERT

In [2]:
def json_print(text):
    # 格式化输出，缩进4个单位
    print(json.dumps(text, sort_keys = True, indent = 4))

In [3]:
def read_data_from_dir(dirname):
    # 文档内容集合
    text = []
    # 遍历文档
    for root, dirs, files in os.walk(dirname):
        for file in files:
            # print(file)
            # 获取文件名
            filename = os.path.splitext(file)[0]
            # 读取文件
            content = json.load(open(root + '/' + file, 'r', encoding = 'utf-8-sig'))
            # 文件名添加到文件中，方便后续生成中间件
            content['FileName'] = filename
            text.append(content)
        
    return text

In [4]:
def keywords_extraction(text):
    r = Rake()
    r.extract_keywords_from_text(text['Text'])
    json_print(text['Text'])
    print("==============================")
    print(r.get_ranked_phrases())
    print("==============================")
    print(r.get_ranked_phrases_with_scores())
    print("==============================")
    print(r.stopwords)
    print("==============================")
    print(r.get_word_degrees())

In [5]:
def yake_it(text):
    language = "en"
    max_ngram_size = 3
    deduplication_thresold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)

    for kw in keywords:
        print(kw)

In [6]:
def get_rank(text):
    r = Rake()
    r.extract_keywords_from_text(text)
    print(r.get_word_degrees())

In [31]:
def bert_it(text):
    model = KeyBERT('distilbert-base-nli-mean-tokens')
    return model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words = 'english', use_maxsum=True, nr_candidates=20, top_n = 10)

In [36]:
sentences = ''
sentences += text[2]['Text']
sentences += text[7]['Text']
sentences += text[12]['Text']
sentences += text[19]['Text']
yake_it(sentences)

('president donald trump', 0.0001618786410693896)
('health', 0.00018860144425106532)
('president trump', 0.000256140016095592)
('coronavirus cases', 0.000290496502101756)
('york city health', 0.00030478933814995)
('coronavirus', 0.0003320549968023795)
('state health department', 0.0003391672858938273)
('york city', 0.00034863000497000683)
('tulsa health department', 0.00041352949780875656)
('world health organization', 0.00044023399925992036)
('city health department', 0.00044367872355489984)
('cases', 0.0004612491174696666)
('health department', 0.0004675250162042261)
('york city mayor', 0.0004956198759993473)
('united states', 0.0005139852510859225)
('state health officials', 0.0005147916150894423)
('white house coronavirus', 0.0005865249150822455)
('johns hopkins university', 0.0006120157458651665)
('state health', 0.0006151640445055714)
('china national health', 0.0006281106730667024)


In [9]:
# 获取词向量
# 该词向量文件形式为：词 空格 词向量，然后换行
# 从http://nlp.stanford.edu/data/glove.6B.zip获取GloVe
word_embeddings = {}
GLOVE_DIR = 'glove.6B.100d.txt'
with open(GLOVE_DIR,encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs

In [10]:
text = read_data_from_dir('news_1')

In [8]:
#eywords_extraction(text[10])

In [9]:
for i in range(20):
    print(i, text[i]['Headline'])

0 China says to have ‘prudent’ oil exploration with Philippines – Manila Bulletin
1 PH to build more cellular sites on Pagasa Island after officials receive ‘welcome’ text messages from China, Vietnam – Manila Bulletin
2 March 30 coronavirus news
3 Malaysia power shift hits China infrastructure drive – Manila Bulletin
4 5 things to know for June 8: George Floyd, police reform, coronavirus, economy, China
5 Why this Japan-China island dispute could be Asia's next military flashpoint
6 February 4 coronavirus news
7 The coronavirus has grounded Chinese tourists. The global travel industry may not recover for years
8 Beijing's crackdown in Xinjiang has separated thousands of children from their parents, new report claims. CNN found two of them
9 The latest on the coronavirus pandemic and vaccines
10 ‘Abominable’ film axed in Malaysia after rebuffing order to cut China map – Manila Bulletin
11 Family of Thai immigrant, 84, says fatal attack 'was driven by hate'
12 The coronavirus pandemic b

In [10]:
sentences = []
# 断句，并写入sentences列表
sentences.append(get_rank_phrases(text[2]['Text']))
sentences.append(get_rank_phrases(text[7]['Text']))
sentences.append(get_rank_phrases(text[12]['Text']))
sentences.append(get_rank_phrases(text[19]['Text']))
# 原数据是好几篇文章，本代码将所有文章的所有句子放在一个列表里，摘要抽取也是基于所有句子（文章）的。
sentences = [y for x in sentences for y in x]
sentences[:10]

['## health ## ## travel ## ## daily life ## new york city',
 '## daily life ## ## business ## around 14 million south korean households',
 '## health ## cnn chief white house correspondent jim acosta asked president trump',
 '## health ## new york city fire department paramedic anthony almojera said',
 '## daily life ## ## travel ## president trump raised questions',
 'weeks .” junior health minister helen whately told bbc radio',
 '## daily life ## michigan governor gretchen whitmer told cnn',
 '## business ## new york city mayor bill de blasio said',
 '.” ## health ## new york city mayor bill de blasio',
 'simple process .” ## business ## us markets opened solidly']

In [13]:
# 文本清洗，去除标点、数字、特殊符号、统一小写
clean_sentences = pd.Series(sentences).str.replace('[^a-zA-Z]', ' ')
clean_sentences = [s.lower() for s in clean_sentences]
clean_sentences[:10]

['   health       travel       daily life    new york city',
 '   daily life       business    around    million south korean households',
 '   health    cnn chief white house correspondent jim acosta asked president trump',
 '   health    new york city fire department paramedic anthony almojera said',
 '   daily life       travel    president trump raised questions',
 'weeks    junior health minister helen whately told bbc radio',
 '   daily life    michigan governor gretchen whitmer told cnn',
 '   business    new york city mayor bill de blasio said',
 '      health    new york city mayor bill de blasio',
 'simple process       business    us markets opened solidly']

In [14]:
stop_words = stopwords.words('english')
def remove_stopwords(str):
    sen = ' '.join([i for i in str if i not in stop_words])
    return sen
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences[:10]

['health travel daily life new york city',
 'daily life business around million south korean households',
 'health cnn chief white house correspondent jim acosta asked president trump',
 'health new york city fire department paramedic anthony almojera said',
 'daily life travel president trump raised questions',
 'weeks junior health minister helen whately told bbc radio',
 'daily life michigan governor gretchen whitmer told cnn',
 'business new york city mayor bill de blasio said',
 'health new york city mayor bill de blasio',
 'simple process business us markets opened solidly']

In [15]:
# 获取每个句子的所有组成词的向量（从GloVe词向量文件中获取，每个向量大小为100），
# 然后取这些向量的平均值，得出这个句子的合并向量为这个句子的特征向量
sentences_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum(
            [word_embeddings.get(w, np.zeros((100,))) for w in i.split()]
        )/(len(i.split()) + 1e-2)
    else:
        v = np.zeros((100,))
    sentences_vectors.append(v)
len(sentences_vectors), len(sentences_vectors[0])

(9028, 100)

In [ ]:
# 计算相似度矩阵，基于余弦相似度
similarity_matrix = np.zeros((len(clean_sentences),len(clean_sentences)))
# 初始化相似度矩阵（全零矩阵）
for i in range(len(clean_sentences)):
    for j in range(len(clean_sentences)):
        if i != j:
            similarity_matrix[i][j] = cosine_similarity(
                sentences_vectors[i].reshape(1,-1), sentences_vectors[j].reshape(1,-1)
            )
similarity_matrix

In [ ]:
# 排序
nx_graph = networkx.from_numpy_array(similarity_matrix)
scores = networkx.pagerank(nx_graph)
# 将相似度矩阵转为图结构
ranked_sentences = sorted(
    ((scores[i],s) for i,s in enumerate(sentences)),reverse=True
)

In [ ]:
# 打印得分最高的前10个句子，即为摘要
for i in range(10):
    print(ranked_sentences[i][1])

In [22]:
help(KeyBERT)

Help on class KeyBERT in module keybert.model:

class KeyBERT(builtins.object)
 |  KeyBERT(model: str = 'distilbert-base-nli-mean-tokens')
 |  
 |  A minimal method for keyword extraction with BERT
 |  
 |  The keyword extraction is done by finding the sub-phrases in
 |  a document that are the most similar to the document itself.
 |  
 |  First, document embeddings are extracted with BERT to get a
 |  document-level representation. Then, word embeddings are extracted
 |  for N-gram words/phrases. Finally, we use cosine similarity to find the
 |  words/phrases that are the most similar to the document.
 |  
 |  The most similar words could then be identified as the words that
 |  best describe the entire document.
 |  
 |  Arguments:
 |      model: the name of the model used by sentence-transformer
 |             for a full overview see https://www.sbert.net/docs/pretrained_models.html
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model: str = 'distilbert-base-nli-mean-tokens'